In [1]:
import pickle
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [2]:
import cafle as cf
from cafle.genfunc import rounding as R
from cafle.genfunc import PY
from cafle.genfunc import EmptyClass

In [3]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 200
print(pd.get_option("display.max_columns"), 
      pd.get_option("display.max_rows"))

40 200


# Introduction

* Logistic center development

# Initial Setting

In [4]:
astn = EmptyClass()

# Input Index Data

In [45]:
tmp_col = ['idxname', 'start', 'periods']
tmp_ary = [['prjt',  '2021-12', 30],
           ['cstrn', '2022-01', 22],
           ['loan',  '2022-01', 26],
           ['sales', '2022-01', 26]]
astn.index = DataFrame(tmp_ary, columns=tmp_col)

idx = cf.PrjtIndex(idxname = list(astn.index.idxname),
                   start   = list(astn.index.start),
                   periods = list(astn.index.periods + 1),
                   freq    = 'M')

# Input Financing Condition Data

In [46]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl']
tmp_ary = [['equity',  1_300,     1_300]]
astn.equity = DataFrame(tmp_ary, columns=tmp_col)

equity = cf.Loan(idx,
                 amt_ntnl = astn.equity.amt_ntnl[0],
                 amt_intl = astn.equity.amt_intl[0])

In [47]:
tmp_col = ['title', 'amt_ntnl', 'amt_intl', 'rate_fee', 'rate_IR']
tmp_ary = [['tra',    65_000,     15_000,      0.015,     0.040],
           ['trb',    20_000,     20_000,      0.035,     0.060],
           ['trc',     5_000,      5_000,      0.070,     0.090]]
astn.loan = DataFrame(tmp_ary, columns=tmp_col)

loan = cf.Intlz_loan(idx, idx.loan,
                     title = astn.loan.title,
                     amt_ntnl = astn.loan.amt_ntnl,
                     amt_intl = astn.loan.amt_intl,
                     rate_fee = astn.loan.rate_fee,
                     rate_IR = astn.loan.rate_IR)

# Input Sales Data

In [48]:
# Pickling Data
with open("data/valuation.pickle", "rb") as fr:
    data_valuation = pickle.load(fr)
valuation = data_valuation['valuation_pivot']
value = valuation.loc[[('23,000', '50,000')]].values[0]

In [49]:
sales = cf.Account(idx, 'Sales')
sales.addamt(idx.sales[0], 125_000)
sales.subscdd(idx.sales[-1], 125_000)

# Input Cost Data

### 1) Set Cost Class

In [50]:
cost = cf.Cost(idx)

### 2) Land Purchase Costs

In [51]:
title = ('lnd', 'prchs') # land purchase cost
byname = '용지매입비'
tmp_ary = [[idx.locval(2021, 12),  1_000],
           [idx.locval(2022,  1), 13_500]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [52]:
title = ('lnd', 'brkrg') # land brokerage fee
byname = '부동산중개비용'
tmp_ary = [[idx.locval(2021, 12),  39],
           [idx.locval(2022,  1),  91]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [53]:
title = ('lnd', 'tax_aqstn') # land acquisition and registration tax
byname = '취등록세'
tmp_ary = [[idx.locval(2022, 1), 675]]
# 토지매매대금 x 취득세 4.0%, 농특세 0.2%, 교육세 0.4%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [54]:
title = ('lnd', 'nhbond') # national housing bond
byname = '국민주택채권'
tmp_ary = [[idx.locval(2022, 1), 36]]
# 토지매매대금 x 매입률 4.5% x 본인부담율 5.5%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [55]:
title = ('lnd', 'jdclcst') # judicial scrivener cost
byname = '법무사비용'
tmp_ary = [[idx.locval(2022, 1), 15]]
# 토지매매대금 x 0.1%
cost.inptcst(title, byname, scdddf=tmp_ary)

### 3) Construction Costs

In [56]:
title = ('cstrn', 'dmltn') # demolition cost
byname = '철거비'
tmp_ary = [[idx.locval(2022, 1), 35],
           [idx.locval(2022, 2), 1_180]]
# 3,550평 x 342천원, 온천공 폐공공사 포함
cost.inptcst(title, byname, scdddf=tmp_ary)

In [57]:
title = ('cstrn', 'civil') # civil engineering
byname = '토목공사'
tmp_ary = [[idx.locval(2022, 3),  849],
           [idx.locval(2022, 4),  3_396],
           [idx.locval(2022, 5),  4_245]]
# 16,327평 x 520천원
cost.inptcst(title, byname, scdddf=tmp_ary)

In [58]:
title = ('cstrn', 'nml_fclt') # normal temperature facilities construction
byname = '상온시설'
ttl_amt = 27_932 # 9,311평 x 3,000천원
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [59]:
title = ('cstrn', 'cld_fclt') # cold temperature facilities construction
byname = '저온시설'
ttl_amt = 33_461 # 8,365평 x 4,000천원
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 4) Indirect Construction Costs

In [60]:
title = ('cstrnidrt', 'wtrelec') # water supply, electricity etc. 
byname = '각종인입비'
tmp_ary = [[idx.locval(2023, 10), 980]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [61]:
title = ('cstrnidrt', 'wsctbn') # water and sewage contribution
byname = '상수도분담금'
tmp_ary = [[idx.locval(2023, 10), 163]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [62]:
title = ('cstrnidrt', 'dsgncst') # design cost
byname = '설계비'
tmp_ary = [[idx.locval(2022, 1), 624]]
# 16,327평 x 38천원
cost.inptcst(title, byname, scdddf=tmp_ary)

In [63]:
title = ('cstrnidrt', 'lcnscst') # license cost
byname = '인허가비용'
tmp_ary = [[idx.locval(2022, 1), 528]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [64]:
title = ('cstrnidrt', 'spvsn') # supervision cost
byname = '감리비'
ttl_amt = 490 # 16,327평 x 30천원, 월별 분할 부과
tmp_idx = idx.cstrn.index
tmp_len = len(idx.cstrn)
tmp_ary = [[x, ttl_amt / tmp_len] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 5) Marketing Costs

In [65]:
title = ('mrktg', 'rentagncy') # rent agency cost
byname = '임대대행수수료'
tmp_ary = [[idx.locval(2024, 3), 517],
           [idx.locval(2024, 4), 517],
           [idx.locval(2024, 5), 517]]
# 임대계약 실행시 월 임대료의 3개월치 지급
# 1개월치 임대료 517백만원 가정
cost.inptcst(title, byname, scdddf=tmp_ary)

In [66]:
title = ('mrktg', 'salescsltg') # sales consulting fee
byname = '매각컨설팅수수료'
tmp_ary = [[idx.locval(2023, 12), 967]]
# 추정매각가 x 0.8%
cost.inptcst(title, byname, scdddf=tmp_ary)

In [67]:
title = ('mrktg', 'advtsmnt') # advertisement and promotion cost
byname = '광고홍보비'
tmp_ary = [[idx.locval(2022, 6), 250],
           [idx.locval(2023, 6), 250]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 6) Tax and Utility bills(제세공과금)

In [68]:
title = ('taxutlt', 'prpttax') # property tax
byname = '재산세종부세'
tmp_ary = [[idx.locval(2022, 6), 37],
           [idx.locval(2023, 6), 37]]
# 5,590 x 0.62% 2년
cost.inptcst(title, byname, scdddf=tmp_ary)

In [69]:
title = ('taxutlt', 'prsvtntax') # preservation registration fee
byname = '보존등기비'
tmp_ary = [[idx.cstrn.index[-1], 3_071]]
# 건물 취득비용 x 3.40%(취득세 2.8%, 농특세 0.2%, 교육세 0.16%, 법무사 0.24%)
cost.inptcst(title, byname, scdddf=tmp_ary)

### 7) Additional Costs

In [70]:
title = ('addtnl', 'pmfee') # Additional expense, PM fee
byname = 'PM수수료'
tmp_ary = [[idx.locval(2021, 12), 200]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [71]:
title = ('addtnl', 'oprtgcst') # company operating cost
unt_amt = 20 # 2,000만원/월
tmp_idx = idx.loan.index
tmp_ary = [[x, unt_amt] for x in tmp_idx]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [72]:
title = ('addtnl', 'rsrvfund') # reserve fund
byname = '예비비'
tmp_ary = [[idx.locval(2022, 6), 907],
           [idx.locval(2023, 6), 907]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [73]:
title = ('addtnl', 'trustcst') # trust fee
byname = '신탁수수료'
tmp_ary = [[idx.locval(2022, 1), 967]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [74]:
title = ('addtnl', 'csltcst') # consulting fee
byname = '사평감평용역비'
tmp_ary = [[idx.locval(2022, 1), 10],
           [idx.locval(2022, 1), 140]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [75]:
title = ('addtnl', 'legalcst') # legal advice fee
byname = '법률자문'
tmp_ary = [[idx.locval(2022, 1), 40]]
cost.inptcst(title, byname, scdddf=tmp_ary)

### 8) Financing Costs

In [76]:
title = ('fncg', 'arngmnt') # financing arrangement fee
byname = '금융주관수수료'
tmp_rate = 0.02 #2.0%
tmp_amt = sum(loan.amt_ntnl)
tmp_ary = [[idx.locval(2022, 1), tmp_amt * tmp_rate]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [77]:
title = ('fncg', 'spcoprtg') # spc operating cost
byname = 'SPC유동화비용'
tmp_ary = [[idx.locval(2022, 1), 66]]
cost.inptcst(title, byname, scdddf=tmp_ary)

In [78]:
title = ('fncg', 'agntbank') # agent banking fee
byname = '대리금융기관'
tmp_ary = [[idx.locval(2022, 1), 30],
           [idx.locval(2023, 1), 30]]
cost.inptcst(title, byname, scdddf=tmp_ary)

# Execution Cash Flow

In [79]:
acc = cf.Intlz_accounts(idx, ['oprtg', 'sales', 'repay'])

In [80]:
# Execute cash flow
for idxno in idx.index:
    #### Loans: set loan withdrawble ####
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno, idx[0])
    loan.tra.set_wtdrbl_intldate(idxno)
    loan.trb.set_wtdrbl_intldate(idxno)
    loan.trc.set_wtdrbl_intldate(idxno)
    
    #### Cash Inflow: cash inflow from sales or rent etc. ####
    salesamt = sales.sub_scdd[idxno]
    if salesamt > 0:
        sales.send(idxno, salesamt, acc.repay)
    
    #### Expected Costs: calculate expected costs ####
    # calculate operating costs
    oprtg_cost = cost['all'].add_scdd[idxno]

    # calculate financial costs
    for trch, loaneach in loan.dct.items():
        if idxno == loaneach.idxfn[0]:
            loaneach.fee.addscdd(idxno, loaneach.fee.amt)
        if all([loaneach.is_wtdrbl, not loaneach.is_repaid]):
            #tmp_ntnl = -loaneach.ntnl.bal_strt[idxno]
            #tmp_IRamt = tmp_ntnl * loaneach.IR.rate_cycle
            loaneach.IR.addscdd(idxno, loaneach.IRamt_topay(idxno))
    fncl_fee = loan.ttl.fee.add_scdd[idxno]
    fncl_IR = loan.ttl.IR.add_scdd[idxno]

    cost_ttl = oprtg_cost + fncl_fee + fncl_IR
    
    
    #### Loans: withdraw loan ####
    # calculate the amount to withdraw
    amt_rqrd = acc.oprtg.amt_rqrd_excess(idxno, cost_ttl)
    
    # withdraw loan amount
    amt_wtdrw = 0
    amt_wtdrw += equity.wtdrw(idxno, equity.amt_intl, acc.oprtg)
    if idxno == idx.loan[0]:
        amt_wtdrw += loan.trc.wtdrw(idxno, loan.trc.amt_intl, acc.oprtg)
        amt_wtdrw += loan.trb.wtdrw(idxno, loan.trb.amt_intl, acc.oprtg)
        amt_wtdrw += loan.tra.wtdrw(idxno, loan.tra.amt_intl, acc.oprtg)
    
    amt_rqrd = max(amt_rqrd - amt_wtdrw, 0)
    amt_rqrd = max(amt_rqrd - loan.trc.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)
    amt_rqrd = max(amt_rqrd - loan.trb.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)
    amt_rqrd = max(amt_rqrd - loan.tra.wtdrw(idxno, amt_rqrd, acc.oprtg), 0)

    
    #### Costs: 토지비, 공사비 등 각종 비용 지출 ####
    for cst_name, cst_acc in cost['all'].dct.items():
        amt_scdd = cst_acc.add_scdd[idxno]
        acc.oprtg.send(idxno, amt_scdd, cst_acc)
        
    
    #### Loans: pay financial cost ####
    acc.oprtg.send(idxno, loan.tra.fee.add_scdd[idxno], loan.tra.fee)
    acc.oprtg.send(idxno, loan.trb.fee.add_scdd[idxno], loan.trb.fee)
    acc.oprtg.send(idxno, loan.trc.fee.add_scdd[idxno], loan.trc.fee)
    
    acc.oprtg.send(idxno, loan.tra.IR.add_scdd[idxno], loan.tra.IR)
    acc.oprtg.send(idxno, loan.trb.IR.add_scdd[idxno], loan.trb.IR)
    acc.oprtg.send(idxno, loan.trc.IR.add_scdd[idxno], loan.trc.IR)
    
    
    #### Loans: repay loan amount ####
    if idxno >= loan.idxfn[-1]: # 만기 도래 여부 확인
        amtrpy = loan.tra.repay_amt(idxno, acc.repay.bal_end[idxno])
        acc.repay.send(idxno, amtrpy, loan.tra.ntnl)
        loan.tra.set_repaid(idxno)
                
        if loan.tra.is_repaid:
            amtrpy = loan.trb.repay_amt(idxno, acc.repay.bal_end[idxno])
            acc.repay.send(idxno, amtrpy, loan.trb.ntnl)
            loan.trb.set_repaid(idxno)
            
        if loan.trb.is_repaid:
            amtrpy = loan.trc.repay_amt(idxno, acc.repay.bal_end[idxno])
            acc.repay.send(idxno, amtrpy, loan.trc.ntnl)
            loan.trc.set_repaid(idxno)
    
    #### Loans : Set back loan unwithdrawble at maturity ####
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    loan.tra.setback_wtdrbl_mtrt(idxno)
    loan.trb.setback_wtdrbl_mtrt(idxno)
    loan.trc.setback_wtdrbl_mtrt(idxno)

In [84]:
R(acc.oprtg.df)

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0.0,0.0,0,"1,300","1,239",61
2022-01-31,0.0,0.0,61,"40,000","23,293","16,768"
2022-02-28,0.0,0.0,"16,768",0,"4,078","12,690"
2022-03-31,0.0,0.0,"12,690",0,"3,747","8,943"
2022-04-30,0.0,0.0,"8,943",0,"6,294","2,649"
2022-05-31,0.0,0.0,"2,649","4,500","7,143",6
2022-06-30,0.0,0.0,6,"4,200","4,107",99
2022-07-31,0.0,0.0,99,"2,900","2,927",72
2022-08-31,0.0,0.0,72,"2,900","2,937",35
2022-09-30,0.0,0.0,35,"3,000","2,946",89


In [88]:
R(cost['all'].df)

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,"1,239",0.0,0,"1,239",0.0,"1,239"
2022-01-31,"21,268",0.0,"1,239","21,268",0.0,"22,507"
2022-02-28,"3,891",0.0,"22,507","3,891",0.0,"26,397"
2022-03-31,"3,560",0.0,"26,397","3,560",0.0,"29,957"
2022-04-30,"6,107",0.0,"29,957","6,107",0.0,"36,063"
2022-05-31,"6,956",0.0,"36,063","6,956",0.0,"43,019"
2022-06-30,"3,905",0.0,"43,019","3,905",0.0,"46,923"
2022-07-31,"2,711",0.0,"46,923","2,711",0.0,"49,634"
2022-08-31,"2,711",0.0,"49,634","2,711",0.0,"52,345"
2022-09-30,"2,711",0.0,"52,345","2,711",0.0,"55,055"


In [94]:
R(loan.ttl.IR.df)

,add_scdd,sub_scdd,bal_strt,amt_add,amt_sub,bal_end
2021-12-31,0,0.0,0,0,0.0,0
2022-01-31,0,0.0,0,0,0.0,0
2022-02-28,188,0.0,0,188,0.0,188
2022-03-31,188,0.0,188,188,0.0,375
2022-04-30,188,0.0,375,188,0.0,562
2022-05-31,188,0.0,562,188,0.0,750
2022-06-30,202,0.0,750,202,0.0,952
2022-07-31,216,0.0,952,216,0.0,"1,169"
2022-08-31,226,0.0,"1,169",226,0.0,"1,395"
2022-09-30,236,0.0,"1,395",236,0.0,"1,631"
